# Homework 1 - Streaming (10 pts)

### DUE: 06/26/2018 before class at 2:00pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_Streaming_NetID.ipynb.

## Task 1 (5 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [1]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

hiredCabs = 0
maxHiredCabs = 0
for row in csvRows('taxi_events.csv'):
    if row['event']=='pickup':
        hiredCabs += 1
        if maxHiredCabs<hiredCabs:
            maxHiredCabs = hiredCabs
    else:
        hiredCabs -= 1
    
print(maxHiredCabs)

250


In [2]:
a = csvRows('taxi_events.csv')

In [3]:
next(a)

OrderedDict([('time', '2015-02-01 00:00:00+00'),
             ('vehicle_id', 'V17131'),
             ('event', 'pickup')])

#### RATIONALE AND JUSTIFICATION

The **csvRows** function defines the generator that reaches each row in the .csv file and reads it, without storing all the data in the file (and therefore minimize the amount of memory usage).

We created two numbers, initially equal to 0: **_hiredCabs_** and **_maxHiredCabs_**; the for loop **adds or substract 1 taxi** from the **_hiredCabs_** number according to _pickup_ or _dropoff_, respectively. If the event is _pickup_, the **maximum number of hired cabs** is being checked - whether the current number of hired cabs had exceeded it. If so, this new-highest number is assigned to the **_maxHiredCabs_** number.

----
## Task 2 (5 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [4]:
import csv
import dateutil 
import pytz


def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

nextDropOffs = []
maxHiredCabs = 0

for row in csvRows('taxi_trips.csv'):
    pickup = dateutil.parser.parse(row['pickup_time'])
    dropOff = dateutil.parser.parse(row['dropoff_time'])
    nextDropOffs.append(dropOff)
    
    nextDropOffs = [i for i in nextDropOffs if i > pickup]
        
    if maxHiredCabs < len(nextDropOffs):
        maxHiredCabs = len(nextDropOffs)


print(maxHiredCabs)

250


#### RATIONALE AND JUSTIFICATION

**csvRows** function  - _as in Task 1_.

This time, all entries have both the _pickup time_ and _dropoff time_, meaning the events of pickups and dropoffs are necessarily not aranged by their order of occurence. Therefore, our task is to check, for each row that is being read in, whether its _pickup time_ happened before of after former cabs' dropoffs. Also, all future dropoffs (meaning all the **dropoffs of active cabs only** for a curtain time) should be stored, but **not after a new cab's _pickup time_ passes it**

In order to efficiently compute the above, I create again the **_maxHiredCabs_** number, initially equal to 0; I also created an empty list of **nextDropOffs** for storing the **active cabs' dropoffs**. The code creates for each entry its _pickup time_ and _dropoff time_ a _datetime variable_, and as a first step stores the new cab's _dropoff time_ to the **nextDropOffs** list. Then the list is redesigned to include only dropoffs that are later then the current entry's pickup time. Here the length of the **nextDropOffs** list at any time is the same as was the **_hiredCabs_** variable at Task 1, thus the **maximum number of hired cabs** is being checked against the length of the list and if it is lower than it, this length is assigned to the **_maxHiredCabs_** number.